# Denoising AE with MNIST (w/ Pytorch)

Recover corrupted MNIST images using a Denoising Autoencoder

## Import PyTorch

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Choose cpu/gpu
use_gpu=1
if (use_gpu):
    print('\nEnable gpu')
    dtype = torch.cuda.FloatTensor
    device = torch.device("cuda") # Uncomment this to run on GPU
    
else:
    print('\nRun on cpu')
    dtype = torch.FloatTensor
    device = torch.device("cpu")

## Initialize Hyper-parameters

If the code takes too long to run you can reduce the number of epochs to 10 or so.

You can also try larger batch sizes.

In [ ]:
num_epochs = 20        # The number of times entire dataset is trained
batch_size = 100       # The size of input data took for one iteration
learning_rate = 0.001  # The speed of convergence

## Download MNIST Dataset

In [ ]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=False)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

## Load the Dataset

We shuffle the loading process of train_dataset to make the learning process independent of data orderness, but the order of test_loader remains to examine whether we can handle unspecified bias order of inputs.

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

## Add noise to data

A function that adds gaussian noise to the data.

In [ ]:
def add_noise(data,scale):
    noise = torch.randn(data.shape) * scale
    noisy_data = data + noise
    return noisy_data


## Look at few uncorrupted images
We'll use the test_loader for this.

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

# Let's see what one test data batch consists of
# --> we have [batch_size] examples of 28x28 pixels in grayscale (i.e. no rgb channels, hence the one). 
print("batch data:", example_data.shape)

# Plot some examples with matplotlib
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  #plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
    
plt.show()

## Exercise: look at corrupted images

Add noise to the previous images using a scale value of:
```python
scale = 0.2
```

Plot the corrupted images.

## Exercise: Autoencoder model

We'll try to remove the noise from the images by using a Denoising Autoencoder:
* Construct an AE with the following structure: 784 (input) -> 350 -> 150 -> 350 -> 784 (output)
* Use ReLU for all hidden layers and Sigmoid for the output layer

## Optional: enable GPU 

In [ ]:
if (use_gpu):
    net.cuda()

## Exercise: Choose the Loss Function and Optimizer

Choose either the Binary Cross Entropy (BCELoss) or Mean Square Error (MSELoss) loss functions.

For the minimization choose the Adam method.

## Exercise: Train the DAE Model

The DAE takes corrupted images as input. The loss is calculated between the output of the network and original (non corrupted) images.

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(num_epochs + 1)]

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)

        corrupted_images = add_noise(images,scale)   
        images = images.type(dtype)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a matrix of 28 x 28 from to a vector of size 784 (view works as numpy's reshape function)
        corrupted_images = Variable(corrupted_images.view(-1, 28*28))
        target = images
        
        # ADD CODE HERE


## Exercise: evaluate the model's training performance

Show in a plot the evolution of the loss as a function of the number of training example seen. 

## Exercise: Test denoising AE

Show the same set of images:
* before corruption
* after corruption
* reconstructed images after DAE correction